In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt # data visualization
import ipywidgets as widgets # interactive widgets
from ipywidgets import Box


In [2]:
df = pd.read_csv('../Data/tracks_cleaned.csv')
df.describe()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,num_artists,year
count,469281.000000,4.692810e+05,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000
mean,27.578334,2.299850e+05,0.044155,0.563647,0.542439,5.224840,-10.200712,0.659208,0.104881,0.449396,0.113413,0.213992,0.552442,118.465459,3.873500,1.295914,1988.594946
std,18.366320,1.270919e+05,0.205439,0.166195,0.251780,3.517928,5.086349,0.473976,0.179904,0.348656,0.266952,0.184386,0.257641,29.784032,0.472858,0.887235,22.813764
min,0.000000,3.344000e+03,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1922.000000
25%,13.000000,1.750530e+05,0.000000,0.453000,0.344000,2.000000,-12.887000,0.000000,0.034000,0.096900,0.000000,0.098300,0.346000,95.549000,4.000000,1.000000,1974.000000
50%,27.000000,2.149070e+05,0.000000,0.577000,0.550000,5.000000,-9.233000,1.000000,0.044300,0.422000,0.000024,0.139000,0.564000,117.363000,4.000000,1.000000,1992.000000
75%,41.000000,2.638000e+05,0.000000,0.686000,0.749000,8.000000,-6.482000,1.000000,0.076400,0.784000,0.009460,0.278000,0.769000,136.335000,4.000000,1.000000,2007.000000
max,98.000000,5.621218e+06,1.000000,0.991000,1.000000,11.000000,5.376000,1.000000,0.971000,0.996000,1.000000,1.000000,1.000000,243.507000,5.000000,58.000000,2021.000000


## Using all features

Since Key is already an estimated value instead of actual we will drop it from the frame. Additonally, we drop release_date due to the inconsisitencies in the data where release_months are only available for some of the data. Since we have already created a column called year in the data cleaning part, we are still able to use that and take advantage of the strong relation between year and popularity

In [3]:
df.drop(['key', 'release_date'],axis=1, inplace=True)

time_signature_df=pd.get_dummies(df["time_signature"])
df = pd.concat([df,time_signature_df],axis=1)
df['mode'] = np.where(df['mode']=='Major', 1, 0)

 Modeling the data + Sets Prep

In [6]:
X= df.loc[:,df.columns !="popularity"] # all the features except popularity
y = df["popularity"] 

In [7]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 469281 entries, 0 to 469280
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   duration_ms       469281 non-null  int64  
 1   explicit          469281 non-null  int64  
 2   danceability      469281 non-null  float64
 3   energy            469281 non-null  float64
 4   loudness          469281 non-null  float64
 5   mode              469281 non-null  int64  
 6   speechiness       469281 non-null  float64
 7   acousticness      469281 non-null  float64
 8   instrumentalness  469281 non-null  float64
 9   liveness          469281 non-null  float64
 10  valence           469281 non-null  float64
 11  tempo             469281 non-null  float64
 12  time_signature    469281 non-null  int64  
 13  num_artists       469281 non-null  int64  
 14  year              469281 non-null  int64  
 15  0                 469281 non-null  uint8  
 16  1                 46

In [9]:
train_scores = []
test_scores = []

for i in range(100):
    # separate the data to training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # save as np.array
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train) 
    y_test = np.array(y_test)

    #creating a regression model
    model_regression = linear_model.LinearRegression()
    model_regression.fit(X_train, y_train)

   # to import to another jupyter notebook
    %store model_regression

    #We estimate models’ success rate in predicting the songs using the score() function which calculates the coefficient of determination. A score which is closer to 1 means the regressor is more accurate.
    train = model_regression.score(X_train, y_train)
    test = model_regression.score(X_test, y_test)

    train_scores.append(train)
    test_scores.append(test)

# Calculate average train and test accuracy
avg_train = sum(train_scores) / len(train_scores)
avg_test = sum(test_scores) / len(test_scores)

print("Average Train Accuracy = " + str(avg_train))
print("Average Test Accuracy = " + str(avg_test))

Stored 'model_regression' (LinearRegression)
Stored 'model_regression' (LinearRegression)
Stored 'model_regression' (LinearRegression)
Stored 'model_regression' (LinearRegression)
Stored 'model_regression' (LinearRegression)
Stored 'model_regression' (LinearRegression)
Stored 'model_regression' (LinearRegression)
Average Train Accuracy = 0.37647194405804696
Average Test Accuracy = 0.3766325089388232


As such, when non-neglgible factors are accounted for, the success rate of a linear regression model in predicting popularity is 0.376 on average after 100 runs

## Dropping Non-essential factors

In [190]:
df = pd.read_csv('../Data/tracks_cleaned.csv')
df.describe()

df.drop(['key', 'release_date', 'num_artists', 'mode', 'speechiness', 'liveness', 'valence', 'tempo', 'time_signature'],axis=1, inplace=True)


In [191]:
X= df.loc[:,df.columns !="popularity"] # all the features except popularity
y = df["popularity"] 

In [192]:
train_scores = []
test_scores = []

for i in range(100):
    # separate the data to training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # save as np.array
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train) 
    y_test = np.array(y_test)

    #creating a regression model
    model_regression = linear_model.LinearRegression()
    model_regression.fit(X_train, y_train)

    #We estimate models’ success rate in predicting the songs using the score() function which calculates the coefficient of determination. A score which is closer to 1 means the regressor is more accurate.
    train = model_regression.score(X_train, y_train)
    test = model_regression.score(X_test, y_test)

    train_scores.append(train)
    test_scores.append(test)

# Calculate average train and test accuracy
avg_train = sum(train_scores) / len(train_scores)
avg_test = sum(test_scores) / len(test_scores)

print("Average Train Accuracy = " + str(avg_train))
print("Average Test Accuracy = " + str(avg_test))

Average Train Accuracy = 0.3746576507012922
Average Test Accuracy = 0.3742977454816405


As seen above, dropping the non-essential features reduces the accuracy by around 0.001

## Using top 6 features from EDA

In [199]:
df = pd.read_csv('../Data/tracks_cleaned.csv')
df.describe()

df.drop(['key', 'release_date', 'num_artists', 'mode', 'speechiness', 'liveness', 'valence', 'tempo', 'time_signature', 'instrumentalness'],axis=1, inplace=True)

In [200]:
X= df.loc[:,df.columns !="popularity"] # all the features except popularity
y = df["popularity"] 

In [201]:
train_scores = []
test_scores = []

for i in range(100):
    # separate the data to training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # save as np.array
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train) 
    y_test = np.array(y_test)

    #creating a regression model
    model_regression = linear_model.LinearRegression()
    model_regression.fit(X_train, y_train)

    #We estimate models’ success rate in predicting the songs using the score() function which calculates the coefficient of determination. A score which is closer to 1 means the regressor is more accurate.
    train = model_regression.score(X_train, y_train)
    test = model_regression.score(X_test, y_test)

    train_scores.append(train)
    test_scores.append(test)

# Calculate average train and test accuracy
avg_train = sum(train_scores) / len(train_scores)
avg_test = sum(test_scores) / len(test_scores)

print("Average Train Accuracy = " + str(avg_train))
print("Average Test Accuracy = " + str(avg_test))

Average Train Accuracy = 0.36808090981203906
Average Test Accuracy = 0.36785937199337787


Reducing the number of features further to important features reduces the success score of the regression model.

## Conclusion

In conclusion, when it comes to regression models, using all the features gives a higher success rate than using only the essential ones. There could be several reasons for this:

1. Even though some factors may have a weaker correlation with popularity, they may still provide some predictive power when used in combination with other factors. The model may be able to extract information from these weaker factors and use it to make more accurate predictions.

2. It's possible that the non-negligible factors alone may not capture all the important information about popularity. There may be other factors that have a weaker correlation individually but when combined with other factors, they provide more useful information about popularity.

Using all factors in a linear regression model could result in a higher success score because the weaker factors may still provide some predictive power, other factors could provide additional useful information, and including more factors could help to reduce multicollinearity issues.



Nonetheless a success score of 0.3-0.4 is a weak score. As such, linear regression models may not be the best in predicting popularity.